In [1]:
import os
import csv
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Stats models 
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer as Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Metrics
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
warnings.filterwarnings('ignore')

##### Tasks
1. Data understanding and preprocessing
2. Create a train and test set (for subsequent machine learning)
3. Data cleaning and feature engineering
   * read up about for feature engineering
4. Visualization
5. Machine Learning

##### Important Features to Notice
`loan_status` = can exclude "Current" or "In Grace Period" since we cannot tell if the loan will be paid or defaulted
`issue_d` = loan issue date (month)
`term` = only 36 or 60 months, can treat as categorical data (can try changing and not changing)

In [3]:
# Reading in the 1.68 GB data file.
df = pd.read_csv("../dataset/lendingclub/accepted_2007_to_2018Q4.csv")

In [ ]:
# Look at the data that are not numerical
df.select_dtypes(include=["object"])

In [5]:
# Remove the last two rows of the data frame.
df = df.iloc[:-2, :]

Identify the rows and features to drop:
1. Drop `loan_status` which are not `Fully Paid` or `Charged Off`
2. Contain high percentage of missing values
3. Multicollinearity
4. Features not associated with loan status

Since we're going to use this dataset to do prediction of whether a loan is fully paid or charged off/defaulted, we can drop the events where the loans are still in place: current and in grace period or late. 

In [ ]:
# Look at cross tabulation between the feature term and target loan_status
pd.crosstab(df["term"], df["loan_status"])

# Subset data frame based on loan status - Charged Off and Fully Paid
mapping = {"Does not meet the credit policy. Status:Charged Off": "Charged Off",
           "Default": "Charged Off",
           "Does not meet the credit policy. Status:Fully Paid": "Fully Paid"}

df["loan_status"] = df["loan_status"].replace(mapping)

df = df[~df["loan_status"].isin(["Current", 
                                 "In Grace Period", 
                                 "Late (16-30 days)",
                                 "Late (31-120 days)"])]

pd.crosstab(df["term"], df["loan_status"])

In [7]:
# Determine the percentage of missing values from each feature.
ms_values_count = df.isnull().sum()
ms_values_perc = 100 * ms_values_count / len(df)

ms_values_df = pd.DataFrame({"ms_values_count": ms_values_count,
                             "ms_values_perc": ms_values_perc})
ms_values_df.sort_values("ms_values_perc", ascending=False, inplace=True)
ms_values_df[ms_values_df["ms_values_perc"] > 0]

# Remove features with more than 50% missing values. 
feat_rm = list(ms_values_df[ms_values_df["ms_values_perc"] > 50].index)
df.drop(feat_rm, axis=1, inplace=True)

# `id` should not be associated with loan status
# `policy_code` only has one value
# `out_prncp` and `out_prncp_inv` mainly just a single value
df.drop(["id", "policy_code", "out_prncp", "out_prncp_inv"], axis=1, inplace=True)

# Drop rows that contain missing values.
df.dropna(axis=0, how="any", inplace=True)

For numerical data, we can check if there are any highly correlated features using Pearson correlation, the default from `pd.DataFrame.corr()`. We can then remove features that are highly correlated (>0.8 on Pearson correlation coefficient). 

Variance Inflation Factor (VIF) gives you a measure of how much the variance of an estimated regression coefficient increases due to the presence of correlated features. 

* VIF > 5 --> the variance of estimated regression coefficient increases largely due to the presence of that feature. 
* VIF = 1 --> the variance of estimated regression coefficient does not increase despite the presence of that feature.

In [8]:
# Check if there is already features that are one-hot encoded.
for col in df.columns:
    if set(df[col].unique()) == {0, 1}:
        print(col)

In [ ]:
# Correlation matrix
corrmat = df.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corrmat, xticklabels=True, yticklabels=True, cmap='RdYlGn');

In [10]:
# Return a list of tuples that contain the pair of highly correlated features.
def find_corr_pairs(df, thresh=0.8):
    corr_pairs = []
    for row in corrmat.index:
        for col in corrmat.columns:
            if (row!=col) and (corrmat.loc[row, col] >= thresh):
                corr_pairs.append((row, col))
    return corr_pairs

# Setting pearson correlation threshold as 0.8
corr_pairs = find_corr_pairs(df, thresh=0.8)

In [ ]:
# Only requires the numerical features.
# Same number of features as used in the correlation matrix.
df_num = df.select_dtypes(include="number")

# This takes a little while as there are >900,000 rows. 
# To calculate the VIF for each feature using a subset of the dataset. 
df_num_subset = df_num.sample(10000)

for i, k in enumerate(df_num_subset.columns):
    print(k, ": ", round(variance_inflation_factor(df_num_subset.values, i), 2), sep="")

There are a number of features with very high VIF values. We shall set the threshold for VIF to be 5, and pick out the columns with VIF value more than the set threshold. 

The columns will then be matched against the correlation list to see if they match.

The correlation matrix can be calculated again to see if there are any more highly correlated features.

In [ ]:
df_num = df.select_dtypes(include="number")

def vif_drop(df, vif_thresh=5):
    
    dropped = True
    feat_to_drop = []

    # VIF is re-calculated each time a feature is dropped.
    # Iteration completed when all VIF < 5.
    while dropped:
        dropped = False
        
        # Select a random subset of sample on each iteration to calculate the VIF
        df_num_subset = df.sample(10000)

        vif_list = []
        for i, k in enumerate(df_num_subset.columns): 
            vif_list.append(round(variance_inflation_factor(df_num_subset.values, i), 3))
        
        # Match the VIF to the feature
        vif_series = pd.Series(vif_list, index=df_num_subset.columns)
        vif_series = vif_series.sort_values(ascending=False)
        
        if vif_series.iloc[0] > vif_thresh:
            
            # Save the features that are dropped
            feat_to_drop.append(vif_series.index[0])
            print("feature dropped: ", vif_series.index[0])
            
            # Drop the feature when the VIF > 5
            df.drop(vif_series.index[0], axis=1, inplace=True)
            dropped = True
    
    return feat_to_drop

feat_to_drop = vif_drop(df_num)

Train a random forest model with just the numerical values to get an idea of what the accuracy score looks like now. 

In [14]:
# Random Forest with just numerical features.
features = df.select_dtypes(include="number").drop(feat_to_drop, axis=1)
target = df["loan_status"]

x_train, x_test, y_train, y_test = train_test_split(features, 
                                                    target, 
                                                    test_size=0.2, 
                                                    random_state=1988)

x_train = pd.DataFrame(x_train, columns=features.columns)
x_test = pd.DataFrame(x_test, columns=features.columns)

rf = RandomForestClassifier(random_state=1988)
rf.fit(x_train, y_train);

y_pred = rf.predict(x_test)

In [ ]:
# Plot the feature importances plot
feat_imp = pd.DataFrame()

feat_imp['Features'] = x_train.columns.values
feat_imp['importance'] = rf.feature_importances_

feat_imp = feat_imp.sort_values(by='importance', ascending=False)
sns.barplot(y="Features", x="importance", data=feat_imp);

The top three features shown to be important with the random forest:
1. `collection_recovery_fee` = post charge off collection fee
2. `last_pymnt_amnt` = last total payment amount received
3. `total_rec_int` = interest received to date

These top two features don't actually seem useful in determining if a person is going to default or pay off his loan since those features will only happen at (or near) the end of the loan period. We can remove those two features and run dataset through the VIF again.

We are also going to drop those features that are not associated with the loan status outcome at all, using a cut-off (arbitrary) of 0.01.

In [ ]:
# First Run
c_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(c_matrix, "\n")
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Drop features that are not associated with the loan status outcome.
feat_to_drop_2 = ["collection_recovery_fee", "last_pymnt_amnt"]
feat_to_drop_3 = list(feat_imp[feat_imp["importance"] < 0.01]["Features"].values)

# Run the VIF again
df_num = (df.select_dtypes(include="number")
            .drop(feat_to_drop_2, axis=1)
            .drop(feat_to_drop_3, axis=1))

feat_to_drop = vif_drop(df_num)

In [ ]:
# Second Run with lesser columns
features = (df.select_dtypes(include="number")
              .drop(feat_to_drop, axis=1)
              .drop(feat_to_drop_2, axis=1)
              .drop(feat_to_drop_3, axis=1))

target = df["loan_status"]

x_train, x_test, y_train, y_test = train_test_split(features, 
                                                    target, 
                                                    test_size=0.2, 
                                                    random_state=1988)

x_train = pd.DataFrame(x_train, columns=features.columns)
x_test = pd.DataFrame(x_test, columns=features.columns)

rf = RandomForestClassifier(random_state=1988)

rf.fit(x_train, y_train);
y_pred = rf.predict(x_test)

# Evaluation Reports
c_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(c_matrix, "\n")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Dropping of the `collection_recovery_fee` and `last_pymnt_amnt` decreases the recall score for the `Charged Off` target further, showing that those two features are indeed more associated with the `Charged Off` target. However, those two features cannot be used in the model as the information are only available at the end of the loan period. 

Apart from training models off the data, there must also be certain domain knowledge, or knowledge of the features to make the selection of features for model training quicker and more accurate. 

In [260]:
# Third Run with StandardScaler()
# Try with StandardScaler for the numerical features
x_train, x_test, y_train, y_test = train_test_split(features, 
                                                    target, 
                                                    test_size=0.2, 
                                                    random_state=1988)

ss = StandardScaler()

x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

x_train = pd.DataFrame(x_train, columns=features.columns)
x_test = pd.DataFrame(x_test, columns=features.columns)

rf = RandomForestClassifier(random_state=1988)

rf.fit(x_train, y_train);

y_pred = rf.predict(x_test)

# Evaluation Reports
c_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(c_matrix, "\n")
print("Classification Report:")
print(classification_report(y_test, y_pred))

After initial run with random forest classifier using just the numerical features, the accuracy of the model seems good at 0.94. All other metrics look decent except for the recall for `Charged Off` loan status. The recall metric of a classification problem is the ratio of number of true positive predictions over the number of actual positive cases. This means out of 100 cases of loan takers who defaulted on the payment, the model is only able to pick out 74 cases. For a loan company, it is not ideal as it means 26 applications for loans are from people who eventually default on their loans. These are considered losses for a loan company. 

The model must be able to identify people who will default on their loans based on the data available as accurately as possible. In contrast with the recall for `Fully Paid` loan status, it is not as crucial to identify people who eventually fully pay off their loans because these are not considered losses for the company. 

Since there are some features that will only be available at the end of the loan period, and there are too many features to sort through, the next version of machine learning for this lending dataset will include only the features more widely used by the community. Listed below. In addition, the categorical features will be included in. 

In [ ]:
# More widely used features
usecols = ["loan_amnt", "term", "int_rate", "installment", "grade", "sub_grade", "emp_title",
           "emp_length", "home_ownership", "annual_inc", "verification_status", "issue_d", "loan_status", "purpose",
           "title", "zip_code", "addr_state", "dti", "earliest_cr_line", "open_acc", "pub_rec",
           "revol_bal", "revol_util", "total_acc", "initial_list_status", "application_type", "mort_acc", "pub_rec_bankruptcies"]